# Testing navier stokes

In [29]:
from PARCtorch.PARCv2 import PARCv2
from PARCtorch.differentiator.differentiator import Differentiator
from PARCtorch.differentiator.finitedifference import FiniteDifference
from PARCtorch.integrator.integrator import Integrator
from PARCtorch.integrator.heun import Heun
from PARCtorch.utilities.unet import UNet

In [12]:
import torch
import matplotlib.pyplot as plt
import os
from glob import glob
import numpy as np
import matplotlib.animation as animation

In [13]:
# Navier Stokes: p, re, u, v
# Adv: u, v
# Dif: u, v
# Poi: p
n_fe_features = 128
unet_ns = UNet([64, 64*2, 64*4, 64*8, 64*16], 4, n_fe_features, up_block_use_concat=[False, True, False, True], skip_connection_indices=[2, 0])
right_diff = FiniteDifference(padding_mode="replicate").cuda()
heun_int = Heun().cuda()
diff_ns = Differentiator(2,                 # 2 state variables: p, re. We always assume 2 velocity being the last 2 channels 
                         n_fe_features,     # Number of features returned by the feature extraction network: 64
                         [2, 3],            # Channel indices to calculate advection: u and v
                         [2, 3],            # Channel indices to calculate diffusion: u and v
                         unet_ns,           # Feature extraction network: unet_burgers
                         "constant",        # Padding mode: constant padding of zero
                         right_diff        # Finite difference method: replication of image_gradients
                         
                        ).cuda()
ns_int = Integrator(True,                     # Clip input data between 0 and 1
                    [(0, 2, 3, 1)],           # Poisson: calculate Poisson on indices 2 (u) and 3 (v), combine it with all the input and output to index 0 (p) 
                    heun_int,                 # Heun integration
                    [None, None, None, None], # No data driven integrator
                    "constant",               # Padding mode: constant padding of zero
                    right_diff,                # Finite difference method: replication of image_gradients
                   ).cuda()
criterion = torch.nn.L1Loss().cuda()

In [23]:
for name, module in model.named_modules():
    print(name)



differentiator
differentiator.list_adv
differentiator.list_adv.2
differentiator.list_adv.2.cdiff
differentiator.list_adv.3
differentiator.list_dif
differentiator.list_dif.2
differentiator.list_dif.3
differentiator.list_mar
differentiator.list_mar.2
differentiator.list_mar.2.spade
differentiator.list_mar.2.spade.leaky_relu
differentiator.list_mar.2.spade.spade1
differentiator.list_mar.2.spade.spade1.initial_conv
differentiator.list_mar.2.spade.spade1.initial_conv.0
differentiator.list_mar.2.spade.spade1.initial_conv.1
differentiator.list_mar.2.spade.spade1.gamma_conv
differentiator.list_mar.2.spade.spade1.beta_conv
differentiator.list_mar.2.spade.conv1
differentiator.list_mar.2.spade.spade2
differentiator.list_mar.2.spade.spade2.initial_conv
differentiator.list_mar.2.spade.spade2.initial_conv.0
differentiator.list_mar.2.spade.spade2.initial_conv.1
differentiator.list_mar.2.spade.spade2.gamma_conv
differentiator.list_mar.2.spade.spade2.beta_conv
differentiator.list_mar.2.spade.conv2
dif

In [20]:
import torch
import os
import logging
from data.dataset import GenericPhysicsDataset, custom_collate_fn, visualize_channels
from torch.utils.data import DataLoader

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(levelname)s] %(message)s')

# Model and paths
model = PARCv2(diff_ns, ns_int, criterion).cuda()
weights_path = '/sfs/qumulo/qhome/jtb3sud/hold/navier_stokes.pt'  # Replace with your model weights path

# Load model weights
if os.path.exists(weights_path):
    model.load_state_dict(torch.load(weights_path))
    logging.info(f"Model weights loaded from {weights_path}")
else:
    logging.error(f"Model weights not found at {weights_path}")
    sys.exit(1)

# Set model to evaluation mode
model.eval()

# Path configurations for datasets
data_dir_test = '/project/vil_baek/data/physics/PARCTorch/NavierStokes/test'    # Replace with your actual test directory path
min_max_path = '/sfs/qumulo/qhome/jtb3sud/PARCtorch/PARCtorch/data/min_max.json'

future_steps = 1
batch_size = 4

# Initialize the dataset and loader for testing
test_dataset = GenericPhysicsDataset(
    data_dirs=[data_dir_test],
    future_steps=future_steps,
    min_max_path=min_max_path
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=1,
    pin_memory=True,
    collate_fn=custom_collate_fn
)

# Testing loop
# Testing loop
logging.info("Starting model evaluation on test data...")

with torch.no_grad():  # Disable gradient computation for evaluation
    for batch_idx, batch in enumerate(test_loader):
        # Assuming batch contains ic, t0, t1, and ground truth (gt)
        ic, t0, t1, gt = batch  # Unpack the batch tuple
        
        # Move the data to the GPU
        ic = ic.cuda()
        t0 = t0.cuda()
        t1 = t1.cuda()
        gt = gt.cuda()

        # Model prediction
        predictions = model(ic, t0, t1)

        # Optionally calculate and log the loss or other metrics
        loss = criterion(predictions[:, :, 1:, :, :], gt[:, :, 1:, :, :])
        logging.info(f"Batch {batch_idx+1}/{len(test_loader)}, Loss: {loss.item()}")

        

logging.info("Model evaluation completed.")


2024-10-07 14:07:46,550 [INFO] Model weights loaded from /sfs/qumulo/qhome/jtb3sud/hold/navier_stokes.pt
2024-10-07 14:07:46,553 [INFO] Starting data validation...
2024-10-07 14:07:46,555 [INFO] Validating .npy files...
Validating files: 100%|██████████| 8/8 [00:00<00:00, 540.29it/s]
2024-10-07 14:07:46,575 [INFO] Data validation completed successfully.
2024-10-07 14:07:46,577 [INFO] Preparing dataset samples...
Listing samples: 100%|██████████| 8/8 [00:00<00:00, 714.70it/s]
2024-10-07 14:07:46,593 [INFO] Total samples in dataset: 304
2024-10-07 14:07:46,596 [INFO] Starting model evaluation on test data...
2024-10-07 14:07:46,712 [INFO] Batch 1/76, Loss: 0.2177707850933075
2024-10-07 14:07:46,764 [INFO] Batch 2/76, Loss: 0.21459530293941498
2024-10-07 14:07:46,814 [INFO] Batch 3/76, Loss: 0.21219569444656372
2024-10-07 14:07:46,853 [INFO] Batch 4/76, Loss: 0.21145108342170715
2024-10-07 14:07:46,891 [INFO] Batch 5/76, Loss: 0.213944673538208
2024-10-07 14:07:46,937 [INFO] Batch 6/76, L